In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report,
    precision_score,
    recall_score,
    accuracy_score,
    f1_score,
)
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np
import pickle

### Read Initial CSVs

In [4]:
Train_df= pd.read_csv("Training_set.csv")
Initial_Test_df= pd.read_csv("Testing_set.csv")

In [5]:
rfc = RandomForestClassifier(class_weight={0: 10, 1: 1})

In [79]:
New_Test_df=pd.read_csv(r"C:\Users\hp\Downloads\archive (6)\flights_sample_3m.csv")

In [80]:
New_Test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Data columns (total 32 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   FL_DATE                  object 
 1   AIRLINE                  object 
 2   AIRLINE_DOT              object 
 3   AIRLINE_CODE             object 
 4   DOT_CODE                 int64  
 5   FL_NUMBER                int64  
 6   ORIGIN                   object 
 7   ORIGIN_CITY              object 
 8   DEST                     object 
 9   DEST_CITY                object 
 10  CRS_DEP_TIME             int64  
 11  DEP_TIME                 float64
 12  DEP_DELAY                float64
 13  TAXI_OUT                 float64
 14  WHEELS_OFF               float64
 15  WHEELS_ON                float64
 16  TAXI_IN                  float64
 17  CRS_ARR_TIME             int64  
 18  ARR_TIME                 float64
 19  ARR_DELAY                float64
 20  CANCELLED                float64
 21  CANCELLA

In [81]:
# Train_df["UniqueCarrier_WN"].value_counts()
word_to_match="UniqueCarrier"
matching_columns = [col for col in Train_df.columns if col.startswith(word_to_match)]
filtered_df=Train_df[matching_columns]
sorted_columns = filtered_df.sum().sort_values(ascending=False).index

for c in sorted_columns:
    print(Train_df[c].value_counts())


UniqueCarrier_WN
0.0    290299
1.0     95217
Name: count, dtype: int64
UniqueCarrier_AA
0.0    327221
1.0     58295
Name: count, dtype: int64
UniqueCarrier_MQ
0.0    338495
1.0     47021
Name: count, dtype: int64
UniqueCarrier_UA
0.0    339826
1.0     45690
Name: count, dtype: int64
UniqueCarrier_OO
0.0    346340
1.0     39176
Name: count, dtype: int64
UniqueCarrier_US
0.0    360199
1.0     25317
Name: count, dtype: int64
UniqueCarrier_DL
0.0    361295
1.0     24221
Name: count, dtype: int64
UniqueCarrier_EV
0.0    363421
1.0     22095
Name: count, dtype: int64
UniqueCarrier_B6
0.0    373284
1.0     12232
Name: count, dtype: int64
UniqueCarrier_AS
0.0    377553
1.0      7963
Name: count, dtype: int64
UniqueCarrier_F9
0.0    378375
1.0      7141
Name: count, dtype: int64
UniqueCarrier_HA
0.0    384368
1.0      1148
Name: count, dtype: int64


In [82]:
word_to_match="Dest"
matching_columns = [col for col in Train_df.columns if col.startswith(word_to_match)]
filtered_df=Train_df[matching_columns]
sorted_columns = filtered_df.sum().sort_values(ascending=False).index

for c in sorted_columns:
    print(Train_df[c].value_counts())


Dest_ORD
0.0    352962
1.0     32554
Name: count, dtype: int64
Dest_DFW
0.0    365958
1.0     19558
Name: count, dtype: int64
Dest_ATL
0.0    367029
1.0     18487
Name: count, dtype: int64
Dest_DEN
0.0    370116
1.0     15400
Name: count, dtype: int64
Dest_LAX
0.0    370997
1.0     14519
Name: count, dtype: int64
Dest_SFO
0.0    373254
1.0     12262
Name: count, dtype: int64
Dest_LAS
0.0    373555
1.0     11961
Name: count, dtype: int64
Dest_PHX
0.0    375468
1.0     10048
Name: count, dtype: int64
Dest_LGA
0.0    377003
1.0      8513
Name: count, dtype: int64
Dest_SLC
0.0    378283
1.0      7233
Name: count, dtype: int64
Dest_MCO
0.0    378503
1.0      7013
Name: count, dtype: int64
Dest_JFK
0.0    378700
1.0      6816
Name: count, dtype: int64
Dest_PHL
0.0    378989
1.0      6527
Name: count, dtype: int64
Dest_SAN
0.0    379110
1.0      6406
Name: count, dtype: int64
Dest_SEA
0.0    379159
1.0      6357
Name: count, dtype: int64
Dest_BOS
0.0    379584
1.0      5932
Name: count, dtype

In [95]:
New_Test_df["FL_DATE"]=pd.to_datetime(New_Test_df["FL_DATE"])
New_Test_df = New_Test_df[New_Test_df['FL_DATE'].dt.year == 2019]
New_Test_df = New_Test_df[New_Test_df['FL_DATE'].dt.month < 7]

In [109]:
#New_Test_df[[New_Test_df["AIRLINE_CODE"]=="WN"]or[New_Test_df["DEST"]=="DFW"]]
cond1=New_Test_df["AIRLINE_CODE"]=="WN"
cond2=((New_Test_df["DEST"]=="LAX")| (New_Test_df["DEST"]=="ATL"))
cond3=((New_Test_df["ORIGIN"]=="ATL") | (New_Test_df["ORIGIN"]=="LAX"))
New_Test_df2=New_Test_df[cond1 & cond2& cond3].iloc[0:10,:]
cols_to_display=["AIRLINE_CODE","DEST","ORIGIN","FL_DATE","DEP_DELAY"]
New_Test_df2[cols_to_display]

,AIRLINE_CODE,DEST,ORIGIN,FL_DATE,DEP_DELAY
3837,WN,LAX,ATL,2019-01-13,5.0
67267,WN,ATL,LAX,2019-01-18,6.0
97560,WN,LAX,ATL,2019-02-10,3.0
215581,WN,LAX,ATL,2019-01-09,11.0
231573,WN,LAX,ATL,2019-06-28,-2.0
246642,WN,LAX,ATL,2019-03-07,27.0
308416,WN,LAX,ATL,2019-05-01,1.0
324113,WN,ATL,LAX,2019-01-26,-1.0
336314,WN,ATL,LAX,2019-02-01,3.0
366488,WN,ATL,LAX,2019-05-19,14.0


In [123]:
Name_Map = {
    'DEST': 'Dest',
    'ORIGIN': 'Origin',
    'AIRLINE_CODE': 'Airline',
    'DEP_DELAY': 'DepDelay',
    'FL_DATE': 'Date'

}
New_Test_df2 = New_Test_df2.rename(columns=Name_Map)
cols_to_display=["Airline","Dest","Origin","Date","DepDelay"]
New_Test_df2[cols_to_display]

,Airline,Dest,Origin,Date,DepDelay
3837,WN,LAX,ATL,2019-01-13,5.0
67267,WN,ATL,LAX,2019-01-18,6.0
97560,WN,LAX,ATL,2019-02-10,3.0
215581,WN,LAX,ATL,2019-01-09,11.0
231573,WN,LAX,ATL,2019-06-28,-2.0
246642,WN,LAX,ATL,2019-03-07,27.0
308416,WN,LAX,ATL,2019-05-01,1.0
324113,WN,ATL,LAX,2019-01-26,-1.0
336314,WN,ATL,LAX,2019-02-01,3.0
366488,WN,ATL,LAX,2019-05-19,14.0


In [205]:
Train_df

,ScheduledArrTime,Distance,ScheduledDepTime,ScheduledElapsedTime,Day,DepTemperature,DepWindSpeed,DepWindDirection,DepPrecipitation,DepRain,...,DayOfWeek_5,DayOfWeek_6,DayOfWeek_7,Month_1,Month_2,Month_3,Month_4,Month_5,Month_6,DepDelay
0,1527,1506,1012,315,22,-0.952000,10.464797,243.435010,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,67
1,1805,457,1740,25,1,22.004500,33.933697,228.871350,0.1,0.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,14
2,1750,491,1605,105,11,6.545500,28.585450,84.217705,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,14
3,1640,733,1500,100,10,12.289000,22.975220,305.433400,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,58
4,1713,2288,1440,153,4,18.052000,4.104631,285.255160,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385511,1536,1187,1045,291,17,9.593500,27.345785,286.049070,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,23
385512,1500,501,1255,125,30,-7.061000,16.657540,276.203340,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,19
385513,2159,1005,1903,176,23,32.086998,12.574260,283.240540,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,81
385514,1930,1172,1700,150,13,6.402000,20.570463,265.985900,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,88


In [218]:
#Unique Carrier List
UniqueCarrier_List=[]
for w in New_Test_df2["Airline"]:
    UniqueCarrier_List.append("UniqueCarrier_"+w)
    
print(UniqueCarrier_List)

#Dest List
Dest_List=[]
for w in New_Test_df2["Dest"]:
    Dest_List.append("Dest_"+w)
    
print(Dest_List)

#Origin List
Origin_List=[]
for w in New_Test_df2["Origin"]:
    Origin_List.append("Origin_"+w)
    
print(Origin_List)

#Month List
Month_List=[]
for w in New_Test_df2["Date"]:
    Month_List.append("Month_"+str(w.month))
    
print(Month_List)


#Day List
Day_List=[]
for w in New_Test_df2["Date"]:
    Day_List.append(w.day)
    
print(Day_List)

#Day of week List
DayofWeek_List=[]
for w in New_Test_df2["Date"]:
    DayofWeek_List.append("DayOfWeek_"+str(w.weekday()))
    
print(DayofWeek_List)

#DepDelay
DepDelay_List=[]
for w in New_Test_df2["DepDelay"]:
    DepDelay_List.append(w)
    
print(DepDelay_List)


#CRSDepTime
CRSDepTime_List=[]
for w in New_Test_df2["CRS_DEP_TIME"]:
    CRSDepTime_List.append(w)
    
print(CRSDepTime_List)


#CRSArrTime
CRSArrTime_List=[]
for w in New_Test_df2["CRS_ARR_TIME"]:
    CRSArrTime_List.append(w)
    
print(CRSArrTime_List)
         
#CRSElapsed_Time
CRSElapsedTime_List=[]
for w in New_Test_df2["CRS_ELAPSED_TIME"]:
    CRSElapsedTime_List.append(w)
    
print(CRSElapsedTime_List)

#Distance
Distance_List=[]
for w in New_Test_df2["DISTANCE"]:
    Distance_List.append(w)
    
print(Distance_List)

['UniqueCarrier_WN', 'UniqueCarrier_WN', 'UniqueCarrier_WN', 'UniqueCarrier_WN', 'UniqueCarrier_WN', 'UniqueCarrier_WN', 'UniqueCarrier_WN', 'UniqueCarrier_WN', 'UniqueCarrier_WN', 'UniqueCarrier_WN']
['Dest_LAX', 'Dest_ATL', 'Dest_LAX', 'Dest_LAX', 'Dest_LAX', 'Dest_LAX', 'Dest_LAX', 'Dest_ATL', 'Dest_ATL', 'Dest_ATL']
['Origin_ATL', 'Origin_LAX', 'Origin_ATL', 'Origin_ATL', 'Origin_ATL', 'Origin_ATL', 'Origin_ATL', 'Origin_LAX', 'Origin_LAX', 'Origin_LAX']
['Month_1', 'Month_1', 'Month_2', 'Month_1', 'Month_6', 'Month_3', 'Month_5', 'Month_1', 'Month_2', 'Month_5']
[13, 18, 10, 9, 28, 7, 1, 26, 1, 19]
['DayOfWeek_6', 'DayOfWeek_4', 'DayOfWeek_6', 'DayOfWeek_2', 'DayOfWeek_4', 'DayOfWeek_3', 'DayOfWeek_2', 'DayOfWeek_5', 'DayOfWeek_4', 'DayOfWeek_6']
[5.0, 6.0, 3.0, 11.0, -2.0, 27.0, 1.0, -1.0, 3.0, 14.0]
[1250, 735, 1910, 1535, 615, 1535, 700, 550, 820, 800]
[1505, 1440, 2120, 1755, 745, 1745, 850, 1305, 1525, 1515]
[315.0, 245.0, 310.0, 320.0, 270.0, 310.0, 290.0, 255.0, 245.0, 255.

In [238]:
Final_Test_df= pd.DataFrame(columns=Train_df.columns)
for i in range(10):
    new_row_data = {UniqueCarrier_List[i]: 1,  Dest_List[i]:1,  Origin_List[i]:1, Month_List[i]:1,"DepDelay":DepDelay_List[i],"Day":Day_List[i],DayofWeek_List[i]:1,
                    "ScheduledDepTime":CRSDepTime_List[i],"ScheduledArrTime":CRSArrTime_List[i],"ScheduledElapsedTime":CRSElapsedTime_List[i],"Distance":Distance_List[i]}
    new_row_df = pd.DataFrame([new_row_data])
    Final_Test_df = pd.concat([Final_Test_df, new_row_df], ignore_index=True)
Final_Test_df=Final_Test_df.fillna(0)
Final_Test_df

C:\Users\hp\AppData\Local\Temp\ipykernel_7280\488500241.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  Final_Test_df = pd.concat([Final_Test_df, new_row_df], ignore_index=True)


,ScheduledArrTime,Distance,ScheduledDepTime,ScheduledElapsedTime,Day,DepTemperature,DepWindSpeed,DepWindDirection,DepPrecipitation,DepRain,...,DayOfWeek_5,DayOfWeek_6,DayOfWeek_7,Month_1,Month_2,Month_3,Month_4,Month_5,Month_6,DepDelay
0,1505,1947.0,1250,315.0,13,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,5.0
1,1440,1947.0,735,245.0,18,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,6.0
2,2120,1947.0,1910,310.0,10,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,3.0
3,1755,1947.0,1535,320.0,9,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,11.0
4,745,1947.0,615,270.0,28,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,-2.0
5,1745,1947.0,1535,310.0,7,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,27.0
6,850,1947.0,700,290.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1.0
7,1305,1947.0,550,255.0,26,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,-1.0
8,1525,1947.0,820,245.0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,3.0
9,1515,1947.0,800,255.0,19,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,14.0


In [239]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
def get_normalized_data(data, dist):
    if dist == "uniform":
        return MinMaxScaler().fit_transform(data)
    elif dist == "norm":
        return StandardScaler().fit_transform(data)
    elif dist == "lognorm":
        return np.log(np.abs(data.flatten()) + 0.1)

In [240]:
columns_distributions_dict = {
        "ScheduledArrTime": "norm",
        "Distance": "lognorm",
        "ScheduledDepTime":"norm",
        "ScheduledElapsedTime":"norm",
        "Day":"uniform",
        "DepTemperature":"norm",
        "DepWindSpeed":"lognorm",
        "DepWindDirection":"uniform",
        "DepPrecipitation":"norm",
        "DepRain":"norm",
        "DepSnowFall":"lognorm",
        "ArrTemperature":"norm",
        "ArrWindSpeed":"lognorm",
        "ArrWindDirection":"uniform",
        "ArrPrecipitation":"norm",
        "ArrRain":"norm",
        "ArrSnowFall":"lognorm"
        
    }
def normalize():

    for column in columns_distributions_dict.keys():
        data = np.array(Final_Test_df[column]).reshape(-1, 1)
        Final_Test_df[column] = get_normalized_data(
            data=data, dist=columns_distributions_dict[column]
        )
    return columns_distributions_dict

In [241]:
normalize()
Final_Test_df

,ScheduledArrTime,Distance,ScheduledDepTime,ScheduledElapsedTime,Day,DepTemperature,DepWindSpeed,DepWindDirection,DepPrecipitation,DepRain,...,DayOfWeek_5,DayOfWeek_6,DayOfWeek_7,Month_1,Month_2,Month_3,Month_4,Month_5,Month_6,DepDelay
0,0.139791,7.574096,0.455426,1.148872,0.444444,0.0,-2.302585,0.0,0.0,0.0,...,0,1,0,1,0,0,0,0,0,5.0
1,-0.026932,7.574096,-0.688693,-1.251756,0.629630,0.0,-2.302585,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,6.0
2,1.717247,7.574096,1.921677,0.977398,0.333333,0.0,-2.302585,0.0,0.0,0.0,...,0,1,0,0,1,0,0,0,0,3.0
3,0.781033,7.574096,1.088580,1.320345,0.296296,0.0,-2.302585,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,11.0
4,-1.809586,7.574096,-0.955284,-0.394389,1.000000,0.0,-2.302585,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,-2.0
5,0.755383,7.574096,1.088580,0.977398,0.222222,0.0,-2.302585,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,27.0
6,-1.540264,7.574096,-0.766449,0.291505,0.000000,0.0,-2.302585,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,1.0
7,-0.373203,7.574096,-1.099688,-0.908809,0.925926,0.0,-2.302585,0.0,0.0,0.0,...,1,0,0,1,0,0,0,0,0,-1.0
8,0.191090,7.574096,-0.499858,-1.251756,0.000000,0.0,-2.302585,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,3.0
9,0.165441,7.574096,-0.544290,-0.908809,0.666667,0.0,-2.302585,0.0,0.0,0.0,...,0,1,0,0,0,0,0,1,0,14.0
